<a href="https://colab.research.google.com/github/samfreund/photonvision/blob/bump-rubik/OBB_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qai_hub torch ultralytics

In [ ]:
!wget https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n-obb.pt

In [ ]:
import qai_hub as hub
import torch
from ultralytics import YOLO
from qai_hub.client import UserError

try:
  hub.set_session_token("y2ag1bqe0jc11ercxzg38gkv0uql64co226fa9n3")
except:
  print("Token already configured, skipping")

# Using pre-trained MobileNet
torch_model = YOLO("/content/yolo11n-obb.pt")
torch_model.model.eval()

# Step 1: Trace model
input_shape = (1, 3, 640, 640)
example_input = torch.rand(input_shape)
with torch.no_grad():
    traced_model = torch.jit.trace(torch_model.model, example_input, strict=False,
    check_inputs=[])

# Step 2: Compile model
compile_job = hub.submit_compile_job(
    model=traced_model,
    device=hub.Device("RB3 Gen 2 (Proxy)"),
    input_specs=dict(image=input_shape),
     options="--target_runtime tflite",
)

# Step 3: Download model
target_model = compile_job.get_target_model()
target_model.download("wheeeee.tflite")